# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [ ]:
# %pip install --upgrade tensorflow[and-cuda]
# %pip install -q huggingface_hub

In [ ]:
import os
os.chdir('..')

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf

In [ ]:
import huggingface_hub
import os
lead_id="AIBridgeEngine/Ben-3B-Lead-v0.1"
bidding_id="AIBridgeEngine/Ben-3B-Bidding-v0.1"
biddinginfo_id="AIBridgeEngine/Ben-3B-Biddinginformation-v0.1"
singledummy_id="AIBridgeEngine/Ben-3B-Singledummy-v0.1"
folder="hfmodels"
huggingface_hub.snapshot_download(repo_id=lead_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=bidding_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=biddinginfo_id, local_dir=folder)
huggingface_hub.snapshot_download(repo_id=singledummy_id, local_dir=folder)

In [ ]:
from nn.leader import Leader
from nn.bid_info import BidInfo
from nn.bidder import Bidder
from nn.lead_singledummy import LeadSingleDummy
lead_suit = Leader(f'{folder}/lead-1000000')
lead_nt = Leader(f'{folder}/lead-1000000')
biddinginfo = BidInfo(f'{folder}/binfo-100000')
bidder = Bidder("", f'{folder}/bidding-100000')
sd_model = LeadSingleDummy(f'{folder}/lr3-1000000')
models = Models(bidder, biddinginfo, lead_suit, lead_nt, sd_model, None, None, 0.1, 0.05, 1.0, 0.1, False, -1, -1, False, True, False, False, 1)
sampler = Sample(0.01, 0.1, 0.03, 0.05, 64, 10, 5000, 5000, 128, 100, 10, 500, False)

In [ ]:
# both vulnerable. you are sitting North as dealer and you hold
hand = 'T54.QT87.A98.943'

# the auction goes:
auction = ["1C","1D","1S","2D","X","PASS","2S","PASS","3C","PASS","3N","PASS","4S",'PASS', 'PASS', 'PASS']

from ddsolver import ddsolver
dds = ddsolver.DDSolver()

# what to lead?
lead_bot = BotLead([True, True], hand, models, sampler, dds, False)
lead = lead_bot.find_opening_lead(auction)

In [ ]:
lead.card

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [ ]:
lead.to_dict()['candidates']

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [ ]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples